### to-do
Currently, this is a copy of Robert Lupton's RGB image demo.
Next: figure out permissions, improve markdown comments

Make true-colour cutouts from HSC data

In [13]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display.rgb as afwRgb
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage



In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13, 8)

In [16]:
dataPath = "/datasets/hsc/repo/rerun/DM-13666/WIDE/"

butler = dafPersist.Butler(dataPath)
skymap = butler.get("deepCoadd_skyMap")

/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/meas_mosaic/16.0+1/python/lsst/meas/mosaic/utils.py:30: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/traitlets/config/application.py

In [9]:
datasettypes = ['calexp', 'calexpBackground', 'icSrc',
               'src', 'srcMatch', 'srcMatchFull', 'ossImage',
               'flattenedImage', 'wcs', 'fcr', 'photoCalib',
               'jointcal_wcs', 'jointcal_photoCalib', 'skyCorr',
               'calexp_camera', 'brightObjectMask', 'deepCoadd_calexp',
               'deepCoadd_det', 'deepCoadd_meas', 'deepCoadd_measMatch',
               'deepCoadd_mergeDet', 'deepCoadd_ref', 'deepCoadd_forced_src',
               'forced_src' ]

In [17]:
help(butler)

Help on Butler in module lsst.daf.persistence.butler object:

class Butler(builtins.object)
 |  Butler provides a generic mechanism for persisting and retrieving data using mappers.
 |  
 |  A Butler manages a collection of datasets known as a repository. Each dataset has a type representing its
 |  intended usage and a location. Note that the dataset type is not the same as the C++ or Python type of the
 |  object containing the data. For example, an ExposureF object might be used to hold the data for a raw
 |  image, a post-ISR image, a calibrated science image, or a difference image. These would all be different
 |  dataset types.
 |  
 |  A Butler can produce a collection of possible values for a key (or tuples of values for multiple keys) if
 |  given a partial data identifier. It can check for the existence of a file containing a dataset given its
 |  type and data identifier. The Butler can then retrieve the dataset. Similarly, it can persist an object to
 |  an appropriate loca

In [ ]:
Select the object of which we want a Pretty Picture.

For coadds the WCS is the same in all bands, but the code handles the general case

In [ ]:
ra, dec, name = 215.9747, -0.4344, "Lens"

raDec = afwCoord.Coord(ra*afwGeom.degrees, dec*afwGeom.degrees)

filters = "grizy"  # filters to process -- we choose our bands when we set B, R, G = ...

cutoutSize = 500   # pixels

Start by finding the tract and patch

In [ ]:
for i, tp in enumerate(skymap.findTractPatchList([raDec])):
    tractInfo, patchInfo = tp
    tract = tractInfo.getId()
    patch = "%d,%d" % patchInfo[0].getIndex()
    print i, tract, patch

Then we can read the desired pixels

In [ ]:
images = {}
cutoutSize = afwGeom.ExtentI(300, 200)

for f in filters:
    filterName = "HSC-%s" % f.upper()
    md = butler.get("deepCoadd_calexp_md", immediate=True,
                    tract=tract, patch=patch, filter=filterName)
    wcs = afwImage.makeWcs(md)
    xy = afwGeom.PointI(wcs.skyToPixel(raDec))

    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

    images[f] = butler.get("deepCoadd_calexp_sub", bbox=bbox, immediate=True,
                            tract=tract, patch=patch, filter=filterName).getMaskedImage()

Generate a RGB images, and optionally write to disk

In [ ]:
rgbFileFmt = "%s-%%s.png" % name if False else None
if not False:
    min = dict(gri=0.01, riz=0.01, izy=0.01)
    max = dict(gri=0.20, riz=0.20, izy=0.25)
else:
    min = dict(gri=0.01, riz=0.01, izy=0.05)
    max = dict(gri=0.20, riz=0.40, izy=0.50)

Q = 10

for bands in ["gri", "riz", "izy"]:
    B, G, R = bands
    rgb = afwRgb.makeRGB(images[R], images[G], images[B],
                         min[bands], max[bands] - min[bands], Q,
                         #saturatedBorderWidth=1, saturatedPixelValue=10
                         )
    
    afwRgb.displayRGB(rgb)
    
    if rgbFileFmt:
        afwRgb.writeRGB(rgbFileFmt % bands, rgb)